<a href="https://colab.research.google.com/github/ananyaagavini/CC_LAB_PES2UG23CS198/blob/main/PES2UG23CS198_Unit1_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi


Wed Jan 28 17:13:49 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   62C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install -q transformers sentencepiece torch


In [3]:
from transformers import pipeline


In [4]:
mask_filler = pipeline(
    "fill-mask",
    model="bert-base-uncased"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 0f0d2ff2-d4c4-4cb3-812a-575dcf06f11d)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: e837c83c-66d7-4338-9964-3fe4faa9a3d3)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json
Retrying in 1s [Retry 1/5].
Device set to use cuda:0


In [5]:
sentence = "The movie was [MASK]."

results = mask_filler(sentence)

for r in results:
    print(r["sequence"], " | score:", round(r["score"], 4))


the movie was good.  | score: 0.1382
the movie was over.  | score: 0.1011
the movie was great.  | score: 0.0532
the movie was perfect.  | score: 0.0283
the movie was amazing.  | score: 0.0204


In [6]:
def suggest_words(sentence, top_k=5):
    results = mask_filler(sentence, top_k=top_k)
    print("Input:", sentence)
    print("Suggestions:\n")

    for i, r in enumerate(results, 1):
        word = r["token_str"].strip()
        score = round(r["score"], 4)
        print(f"{i}. {word}  (confidence: {score})")

suggest_words("The movie was [MASK].")


Input: The movie was [MASK].
Suggestions:

1. good  (confidence: 0.1382)
2. over  (confidence: 0.1011)
3. great  (confidence: 0.0532)
4. perfect  (confidence: 0.0283)
5. amazing  (confidence: 0.0204)


In [7]:
suggest_words("The forest was [MASK].")
suggest_words("The sound was [MASK].")


Input: The forest was [MASK].
Suggestions:

1. silent  (confidence: 0.1453)
2. dark  (confidence: 0.1352)
3. quiet  (confidence: 0.1019)
4. gone  (confidence: 0.0534)
5. empty  (confidence: 0.0472)
Input: The sound was [MASK].
Suggestions:

1. deafening  (confidence: 0.0898)
2. familiar  (confidence: 0.0558)
3. loud  (confidence: 0.0532)
4. unmistakable  (confidence: 0.034)
5. horrible  (confidence: 0.0278)


In [9]:
import re

def suggest_words(sentence, top_k=10):
    if "[MASK]" not in sentence:
        print("⚠️ Please include [MASK] in the sentence.")
        return

    results = mask_filler(sentence, top_k=top_k)

    print("\nInput:", sentence)
    print("Suggestions:\n")

    count = 1
    for r in results:
        word = r["token_str"].strip()

        # filter punctuation-only outputs
        if re.fullmatch(r"\W+", word):
            continue

        score = round(r["score"], 4)
        print(f"{count}. {word}  (confidence: {score})")
        count += 1

        if count > 5:
            break

while True:
    text = input("\nEnter a sentence with [MASK] (or type exit): ")
    if text.lower() == "exit":
        break

    # auto-fix missing period
    if text.strip().endswith("[MASK]"):
        text = text.strip() + "."

    suggest_words(text)




Enter a sentence with [MASK] (or type exit): The room is very [MASK]

Input: The room is very [MASK].
Suggestions:

1. small  (confidence: 0.1979)
2. large  (confidence: 0.1062)
3. quiet  (confidence: 0.0903)
4. dark  (confidence: 0.0798)
5. warm  (confidence: 0.0492)

Enter a sentence with [MASK] (or type exit): This is place looks [MASK]

Input: This is place looks [MASK].
Suggestions:

1. good  (confidence: 0.1968)
2. familiar  (confidence: 0.1408)
3. like  (confidence: 0.089)
4. amazing  (confidence: 0.0637)
5. great  (confidence: 0.0503)

Enter a sentence with [MASK] (or type exit): The sky is [MASK]

Input: The sky is [MASK].
Suggestions:

1. clear  (confidence: 0.201)
2. blue  (confidence: 0.1899)
3. dark  (confidence: 0.1326)
4. gray  (confidence: 0.0625)
5. black  (confidence: 0.0546)


KeyboardInterrupt: Interrupted by user

# Observations – Synonym Suggester using BERT (fill-mask)

### 1. Context-aware word prediction
BERT does not suggest words based on dictionary meaning alone. The predictions change depending on the surrounding words.  
For example, “The movie was [MASK].” produced words related to opinions (great, boring, amazing), while “The forest is [MASK].” produced more environment-related adjectives.

---

### 2. Importance of right-side context
When the input ended exactly at the masked token (e.g., “The room is [MASK]”), the model mostly predicted punctuation such as “.”, “!”, and “?”.  
This showed that without right-side context, BERT assumes the sentence is ending and predicts sentence terminators instead of descriptive words.

After adding minimal right context (e.g., “The room is [MASK].” or “The room is very [MASK].”), the model began producing meaningful adjectives like “big,” “clean,” and “dark.”

---

### 3. Sensitivity to sentence structure
Small changes in sentence structure significantly affected predictions.  
Adding words like “very,” “looks,” or “feels” before the mask changed the type of adjectives suggested, indicating that BERT strongly uses grammatical cues to infer the role of the missing word.

---

### 4. Confidence-ranked outputs
The model outputs multiple candidate words along with confidence scores.  
Higher-ranked predictions were usually more natural and grammatically correct, while lower-ranked ones were less relevant, showing how probability is distributed across possible words.

---

### 5. Encoder-only model behavior
Since BERT is an encoder-only model trained using Masked Language Modeling, it performed very well at predicting missing words but cannot generate long new text sequences like generative models (e.g., GPT or BART).

---

### 6. Practical insight
This experiment demonstrated that masked language models rely heavily on bidirectional context. They infer not only meaning, but also whether the missing token should be a word, symbol, or punctuation based on learned language patterns.
